<a href="https://colab.research.google.com/github/bhadreshpsavani/EfficientQAExperiments/blob/master/NaturalQAT5TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 16.4MB/s 
     |████████████████████████████████| 1.1MB 26.3MB/s 
     |████████████████████████████████| 3.0MB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6560ff8fa7f6cbe8b4847fd59f2ab8abde310922d6482661acd82beab71821fb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 1.1MB 15.6MB/s 
     |████████████████████████████████| 890kB 14.5MB/s 
     |████████████████████████████████| 3.0MB 34.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=82e2e090cdd56af9997975f6f3833f606f5605b08c89ea1da7b5cff800983b42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import re
import json
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import T5Tokenizer, TFT5ForConditionalGeneration, T5Config

max_len = 512
configuratio = T5Config()  # default parameters and configuration for BERT

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small', eos_token='</s>', sep_token='<sep>')

In [ ]:
!git clone https://github.com/efficientqa/nq-open.git

Cloning into 'nq-open'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [ ]:
train_df = pd.read_json("nq-open/NQ-open.train.jsonl", orient='columns', lines=True)
valid_df = pd.read_json("nq-open/NQ-open.dev.jsonl", orient='columns', lines=True)

In [ ]:
train_df.head()

,answer,question
0,[Fernie Alpine Resort],where did they film hot tub time machine
1,[Neither vessel],who has the right of way in international waters
2,[Marley],who does annie work for attack on titan
3,"[November 6, 1986]",when was the immigration reform and control ac...
4,[1950],when was puerto rico added to the usa


In [ ]:

# class SquadExample:
#     def __init__(self, question, answer_text, tokenize=tokenizer):
#         self.question = question
#         self.answer_text = answer_text
#         self.max_len = 512
#         self.tokenizer = tokenizer

#     def preprocess(self):
#         question = self.question
#         answer_text = self.answer_text

#         # Clean context, answer and question
#         question = " ".join(str(question).split())
#         answer = " ".join(str(answer_text).split())
#         question = 'nq question: '+' '.join(question.split())+"? </s>"
#         answer = ' <sep> '.join(answer) + " </s>"
#         answer = ' '.join(answer.split())

#         source = self.tokenizer.batch_encode_plus(
#             [question], 
#             max_length= self.max_len,
#             add_special_tokens=True,
#             return_special_tokens_mask=True,
#             truncation=True, 
#             pad_to_max_length=True,
#             return_tensors='tf')
        
#         target = self.tokenizer.batch_encode_plus(
#             [answer], 
#             max_length= self.max_len, 
#             add_special_tokens=True,
#             return_special_tokens_mask=True,
#             truncation=True,
#             pad_to_max_length=True,
#             return_tensors='tf')

#         self.source_ids = source['input_ids']
#         self.source_mask = source['attention_mask']
#         self.target_ids = target['input_ids']


# def create_squad_examples(df):
#     print("\nCreating Examples")
#     squad_examples = []
#     for ind in tqdm(df.index):
#         squad_eg = SquadExample(
#             df["question"][ind], df["answer"][ind]
#         )
#         squad_eg.preprocess()
#         squad_examples.append(squad_eg)
#     return squad_examples


# def create_inputs_targets(squad_examples):
#     dataset_dict = {
#         "source_ids": [],
#         "source_mask": [],
#         "target_ids": [],
#         "target_mask": []
#     }
#     print("\nCreating final Target")

#     for item in tqdm(squad_examples):
#         for key in dataset_dict:
#             dataset_dict[key].append(getattr(item, key))

#     x = [
#         np.array(dataset_dict["source_ids"]),
#         np.array(dataset_dict["source_mask"]),
#     ]
#     y = np.array(dataset_dict["target_ids"])
#     return x, y

In [82]:
def preprocess(row):
  question = row['question']
  answer = row['answer']

  # Clean context, answer and question
  question = 'nq question: '+question+"?</s>"
  answer = ' <sep> '.join(answer) + "</s>"

  source = tokenizer.batch_encode_plus(
      [question], 
      max_length=256,
      add_special_tokens=True,
      return_special_tokens_mask=True,
      truncation=True, 
      pad_to_max_length=True,
      return_tensors='tf')
  
  target = tokenizer.batch_encode_plus(
      [answer], 
      max_length=128, 
      add_special_tokens=True,
      return_special_tokens_mask=True,
      truncation=True,
      pad_to_max_length=True,
      return_tensors='tf')

  return {'inputs': np.array(source['input_ids']), 'attention_mask': np.array(source['attention_mask'])}, [np.array(target['input_ids'])]

In [83]:
input = train_df[:100].apply(preprocess, axis=1)

In [84]:
train_X, train_y = (input[0]), (input[1])

In [85]:
train_X[0]

{'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32),
 'inputs': array([[   3,   29, 1824,  822,   10,  213,  410,   79,  814, 1312, 6244,
     

In [ ]:
# train_squad_examples = create_squad_examples(train_df[:100])
# x_train, y_train = create_inputs_targets(train_squad_examples)
# print(f"\n{len(train_squad_examples)} training points created.")

  1%|          | 1/100 [00:00<00:12,  7.80it/s]


Creating Examples


100%|██████████| 100/100 [00:00<00:00, 79891.50it/s]



Creating final Target

100 training points created.


In [ ]:
# eval_squad_examples = create_squad_examples(valid_df[:100])
# x_eval, y_eval = create_inputs_targets(eval_squad_examples)
# print(f"{len(eval_squad_examples)} evaluation points created.")

 21%|██        | 21/100 [00:00<00:00, 204.03it/s]


Creating Examples


100%|██████████| 100/100 [00:00<00:00, 151364.27it/s]



Creating final Target
100 evaluation points created.


In [68]:

def create_model():
    ## BERT encoder
    embedding = TFT5ForConditionalGeneration.from_pretrained("t5-base")

    ## QA Model
    input_ids = layers.Input(shape=(256,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(256,), dtype=tf.int32)

    embedding = embedding(
        inputs=input_ids, decoder_input_ids = input_ids, attention_mask=attention_mask, decoder_attention_mask = attention_mask
    )[0]

    output_ids = layers.Input(shape=(128,), dtype=tf.int32)

    model = keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=[output_ids],
    )
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer)
    return model


In [69]:
model = create_model()


If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [65]:
# use_tpu = True
# if use_tpu:
#     # Create distribution strategy
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)

#     # Create model
#     with strategy.scope():
#           model = create_model()
# else:
#     model = create_model()

# model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.44.37.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.44.37.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)

If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ValueError: ignored

In [ ]:
# optimizer = keras.optimizers.Adam(lr=5e-5)
# model.compile(optimizer=optimizer)

In [86]:
row={
    "question":"where is it?",
     "answer":"It is everywhere."
}
inputs,output_ids= preprocess(row)
print(inputs)
print(output_ids)

{'inputs': array([[   3,   29, 1824,  822,   10,  213,   19,   34, 8546,    1,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  

In [87]:
model.fit(
    x=inputs,
    y=output_ids,
    epochs=1,  
    # verbose=2,
    # batch_size=1,
)

AssertionError: ignored